In [1]:
# -*- coding: utf-8 -*-

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import tensorflow as tf
import numpy as np
import time

from rnn_model import Model
from data_loader import text_data
data = text_data("./dataset/ptb")


def initialize_session():
    config = tf.ConfigProto()
    #config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)

##################################################
BATCH_SIZE = 20         # 배치 사이즈
num_k = 7               # 앞에 볼 단어 개수
emb_dim = 64            # 단어 embedding dimension
hidden_dim = 128        # RNN hidden dim
learning_rate = 0.001   # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
##################################################

model = Model(num_k=num_k, emb_dim=emb_dim, hidden_dim=hidden_dim, vocab_size=data.vocab_size,
              use_clip=True, learning_rate=learning_rate)

sess = initialize_session()
sess.run(tf.global_variables_initializer())


def sample_test(test_input=""):
    # test_input = raw_input("test text: ") # input("test text: ") for python 2, 3
    test_x = np.zeros((1, num_k), dtype=np.int32)
    words = test_input.split()
    for i in range(min(num_k, len(words))):
        test_x[0][-i-1] = data.w2idx[words[-i-1]]
    out_x = sess.run(model.out_y, feed_dict={model.x: test_x})
    print(out_x[0], data.idx2w[out_x[0]])

def test_model():
    num_it = int(len(data.test_ids) / BATCH_SIZE)
    test_x = np.zeros((BATCH_SIZE, num_k), dtype=np.int32)
    mask = np.zeros(BATCH_SIZE, dtype=np.int32)
    test_loss, test_cnt = 0, 0

    for _ in range(num_it):
        test_ids, length = data.get_test(BATCH_SIZE)
        max_len = max(length)

        test_x.fill(0)
        mask.fill(0)

        for i in range(num_k - 1, max_len - 2):
            for batch in range(len(test_ids)):
                for j in range(0, num_k):
                    if i < j or i - j >= length[batch]:
                        break
                    test_x[batch][num_k - j - 1] = test_ids[batch][i - j]
                mask[batch] = 1 if length[batch] > i + 1 else 0

                if length[batch] > i + 1:
                    input_y[batch] = test_ids[batch][i + 1]

            loss = sess.run(model.loss, feed_dict={model.x: test_x, model.y: input_y, model.mask: mask})
            test_loss += loss
            test_cnt += 1
    print("test loss: {:.3f}".format(test_loss / test_cnt))


input_x = np.zeros((BATCH_SIZE, num_k), dtype=np.int32)
input_y = np.zeros(BATCH_SIZE, dtype=np.int32)
input_mask = np.zeros(BATCH_SIZE, dtype=np.int32)
length = np.zeros(BATCH_SIZE, dtype=np.int32)

avg_loss, it_cnt = 0, 0
it_log, it_test, it_save, it_sample = 10, 200, 1000, 200
start_time = time.time()

for it in range(0, 1):
    train_ids, length = data.get_train(BATCH_SIZE)
    max_len = max(length)
    input_x.fill(0)
    print(train_ids)
    print('============================')
    for i in range(num_k - 1, max_len - 2):
        for batch in range(len(train_ids)):
            for j in range(0, num_k):
                if i < j or i-j >= length[batch]:
                    break
                input_x[batch][num_k-j-1] = train_ids[batch][i-j]
            input_mask[batch] = 1 if length[batch] > i + 1 else 0

            if length[batch] > i+1:
                input_y[batch] = train_ids[batch][i+1]
        print(input_x)
        print(input_y)
        loss, _,x_emb = sess.run([model.loss, model.update, model.x_emb],
                           feed_dict={model.x: input_x, model.y: input_y, model.mask: input_mask})
 
        avg_loss += loss
        it_cnt += 1
        print('x_emb ' ,x_emb.shape)

    if it % it_log == 0:
        print(" it: {:4d} | loss: {:.3f} - {:.2f}s".format(it, avg_loss / it_cnt, time.time() - start_time))
        avg_loss, it_cnt = 0, 0

    if it % it_test == 0 and it > 0:
        test_model()
    if it % it_save == 0 and it > 0:
        model.save(sess)
    if it % it_sample == 0 and it > 0:
        sample_test(test_input="again the specialists were not able to")

#sess.close()

/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[list([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
 list([25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 27])
 list([39, 26, 40, 41, 42, 26, 43, 32, 44, 45, 46])
 list([47, 26, 27, 28, 29, 48, 49, 41, 42, 50, 51, 52, 53, 54, 55, 35, 36, 37, 42, 56, 57, 58, 59])
 list([35, 60, 42, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 35, 71, 72, 42, 73, 74, 75, 35, 46, 42, 76, 77, 64, 78, 79, 80, 27, 28, 81, 82, 83])
 list([32, 61, 84, 26, 40, 85, 26, 62, 78, 86, 32, 26, 87, 88, 89, 90, 64, 78, 91, 92, 93, 94, 95, 96, 97, 82, 98])
 list([26, 99, 32, 100, 42, 101, 102, 26, 103, 93, 104, 66, 105, 106, 107, 26, 108, 109, 26, 67, 68, 108, 27])
 list([110, 111, 112, 113, 83, 79, 80, 35, 114, 81, 32, 115, 116, 117, 108, 118, 119, 101, 120, 121, 42, 122, 35, 123, 124, 64, 125, 101, 126, 64, 32, 127])
 list([35, 26, 26, 98, 56, 40, 128, 29, 129])
 list([130, 131, 132, 133, 28, 81, 134, 135, 136, 42, 61, 137, 138, 139, 140])
 list([141, 40, 142, 61, 108, 14

In [ ]:
BATCH_SIZE = 20         # 배치 사이즈
num_k = 7               # 앞에 볼 단어 개수
emb_dim = 64            # 단어 embedding dimension
hidden_dim = 128        # RNN hidden dim
learning_rate = 0.001   # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부

In [7]:
emb_W

NameError: name 'emb_W' is not defined